# IDEFICS: A Flamingo-based model, trained at scale for the community
# Finetuning Demo Notebook:

<div style="text-align: center;">
</div>
<div style="display: flex; justify-content: center;">
    <img src="https://huggingface.co/HuggingFaceM4/idefics-80b/resolve/main/assets/Idefics_colab.png" alt="Idefics image" >
</div>

Credit: [Flamingo blog](https://www.deepmind.com/blog/tackling-multiple-tasks-with-a-single-visual-language-model)

This google colab notebook shows how to run predictions with the 4-bit quantized 🤗 [Idefics-9B model](https://huggingface.co/HuggingFaceM4/idefics-9b) and finetune it on a specific dataset.

[IDEFICS](https://huggingface.co/HuggingFaceM4/idefics-80b) is a multi-modal model based on the [Flamingo](https://arxiv.org/abs/2204.14198) architecture. It can take images and texts as input and return text outputs but it does not support image generation. \\
IDEFICS is built on top of two unimodal open-access pre-trained models to connect the two modalities. Newly initialized parameters in the form of Transformer blocks bridge the gap between the vision encoder and the language model. The model is trained on a mixture of image/text pairs and unstrucutred multimodal web documents. \\
The [finetuned versions](https://huggingface.co/HuggingFaceM4/idefics-80b-instruct) of IDEFICS behave like LLM chatbots while also understanding visual input. \\
You can play with the [demo here](https://huggingface.co/spaces/HuggingFaceM4/idefics_playground)

The code for this notebook was contributed to by *Léo Tronchon, Younes Belkada, and Stas Bekman*, the IDEFICS model has been contributed to by: *Lucile Saulnier, Léo Tronchon, Hugo Laurençon, Stas Bekman, Amanpreet Singh, Siddharth Karamcheti, and Victor Sanh*

In [1]:
! gdown 1TSYIdXmhFxvjoclZuqcMrvcYXXFk0w6W

Downloading...
From (original): https://drive.google.com/uc?id=1TSYIdXmhFxvjoclZuqcMrvcYXXFk0w6W
From (redirected): https://drive.google.com/uc?id=1TSYIdXmhFxvjoclZuqcMrvcYXXFk0w6W&confirm=t&uuid=a4662a3d-948e-4d3c-8f2e-fdc3b0ba5370
To: /content/visualgenome_AR.csv
100% 209M/209M [00:02<00:00, 87.2MB/s]


In [2]:
! wget --no-clobber https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip
! wget --no-clobber https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip

--2024-03-18 14:49:46--  https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9731705982 (9.1G) [application/zip]
Saving to: ‘images.zip’

images.zip          100%[===================>]   9.06G  7.21MB/s    in 20m 44s 

2024-03-18 15:10:31 (7.46 MB/s) - ‘images.zip’ saved [9731705982/9731705982]

--2024-03-18 15:10:31--  https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5471658058 (5.1G) [application/zip]
Saving to: ‘images2.zip’

images2.zip         100%[===================>]   5.10G  8.13MB/s    in 17m 13s 

2024-03-18 15:27:45 (5.05 MB/s) - ‘images2.zip’ saved [5471658058/5471658058]


In [3]:
! unzip -o -q images.zip
! unzip -o -q images2.zip

In [4]:
!mkdir /content/images
!find /content/VG_100K -type f -exec mv -t /content/images/ {} +
!find /content/VG_100K_2 -type f -exec mv -t /content/images/ {} +

# Install and import necessary libraries

In [5]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git@add-model-idefics
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × git checkout -q add-model-idefics did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q add-model-idefics did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

# Load quantized model
First get the quantized version of the model. This will allow us to use the 9B version of Idefics with a single 16GB gpu



In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# checkpoint = "HuggingFaceM4/tiny-random-idefics"
checkpoint = "HuggingFaceM4/idefics-9b"

# Here we skip some special modules that can't be quantized properly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

processor = AutoProcessor.from_pretrained(checkpoint, token=True)
# Simply take-off the quantization_config arg if you want to load the original model
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")

preprocessor_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

If you print the model, you will see that all `nn.Linear` layers are in fact replaced by `bnb.nn.Linear4bit` layers.

In [ ]:
# print(model)

# Inference
Let's make a simple method to test the model's inference

In [8]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)


Let's run prediction with the quantized model for the image below which pictures two kittens. \\
<img src="https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg" width="400"/>

In [38]:
url = "https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg"
prompts = [
    # "Instruction: provide an answer to the question. Use the image to answer.\n",
    url,
    "Question: What's on the picture? Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=100)

Question: What's on the picture? Answer: Two kittens.


Now let's see how the model fares on pokemon knowledge before we try to finetune it further. \\
<img src="https://images.pokemontcg.io/pop6/2_hires.png" width="194"/>


In [35]:
# check generation before finetuning

prompts = [
    "https://sb.ecobnb.net/app/uploads/sites/3/2021/09/Progetto-senza-titolo-5.jpg.webp",
    "Question: What's on the picture? Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=100)

Question: What's on the picture? Answer: A turtle and fishes.


# Finetuning dataset
Prepare the dataset that will be used for finetuning


In [11]:
import pandas as pd
df = pd.read_csv("visualgenome_AR.csv")

In [13]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [14]:
# load dataset
dataset = dataset.train_test_split(test_size=0.002)
train_ds = dataset["train"]
eval_ds = dataset["test"]

In [15]:
def convert_to_rgb(image):
    # `image.convert("RGB")` would only work for .jpg images, as it creates a wrong background
    if image.mode == "RGB":
        return image

    # For transparent images, call to `alpha_composite` handles it's case
    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    alpha_composite = alpha_composite.convert("RGB")
    return alpha_composite

In [16]:
def ds_transforms(example_batch):
    image_size = processor.image_processor.image_size
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std

    image_transform = transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC), # Bicubic interpolation
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean, std=image_std),
    ])

    prompts = []
    for i in range(len(example_batch['question_en'])):

        prompts.append(
            [
                example_batch['image_path'][i],
                f"Question: {example_batch['question_en'][i]} Answer: {example_batch['answer_en'][i]}.</s>",
            ],
        )

    inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)

    inputs["labels"] = inputs["input_ids"]

    return inputs

In [17]:
# load and prepare dataset
train_ds.set_transform(ds_transforms)
eval_ds.set_transform(ds_transforms)

# LoRA
After specifying the low-rank adapters (LoRA) config, we load the PeftModel using the get_peft_model utility function

In [18]:
model_name = checkpoint.split("/")[1]
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)
model = get_peft_model(model, config)

In [19]:
model.print_trainable_parameters()

trainable params: 19,750,912 || all params: 8,949,430,544 || trainable%: 0.2206946230030432


# Training
Finally, using the Hugging Face Trainer, we can finetune the model!

For the sake of the demo, we have set the max_steps at 40. That's about 0.05 epoch on this dataset, so feel free to tune further!

It has been reported that fine-tuning in mixed precision fp16 can lead to overflows. As such, we recommend training in mixed precision bf16 when possible.

In [20]:
train_ds

Dataset({
    features: ['question_en', 'answer_en', 'image_id', 'qa_id', 'image_path', 'question_ar', 'answer_ar'],
    num_rows: 1442431
})

In [25]:
training_args = TrainingArguments(
    output_dir=f"{model_name}-visualgenome",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=40,
    eval_steps=20,
    logging_steps=20,
    max_steps=40,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    load_best_model_at_end=True,
    report_to=None,
    optim="paged_adamw_8bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
)

trainer.train()

Step,Training Loss,Validation Loss
20,1.059600,1.043456


Step,Training Loss,Validation Loss
20,1.059600,1.043456
40,1.039500,1.034224


TrainOutput(global_step=40, training_loss=1.0495121955871582, metrics={'train_runtime': 1072.8376, 'train_samples_per_second': 1.79, 'train_steps_per_second': 0.037, 'total_flos': 3526538387174208.0, 'train_loss': 1.0495121955871582, 'epoch': 0.0})

In [36]:
prompts = [
    # "Instruction: provide an answer to the question. Use the image to answer.\n",
    '/content/test.png',
    "Question: What's on the picture? Answer:",
]

# check generation again after finetuning
check_inference(model, processor, prompts, max_new_tokens=500)

/content/test.pngQuestion: What's on the picture? Answer: A man..


# Push your new model to the hub!


In [32]:
model.push_to_hub(f"{model_name}-visualgenome", private=False)

adapter_model.safetensors:   0%|          | 0.00/79.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/messawey/idefics-9b-visualgenome/commit/c42c447879e7858425231d008009f657d284d1f8', commit_message='Upload model', commit_description='', oid='c42c447879e7858425231d008009f657d284d1f8', pr_url=None, pr_revision=None, pr_num=None)